In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
import random

# API Connection
from sodapy import Socrata

client = Socrata("data.cityofnewyork.us", "yUnYRTjXHWdK6ATxbXOsJR52v", timeout=60)

# First 20,000 results, returned as JSON from API / converted to Python list of dictionaries by sodapy.
total_records = 8200000  # Total number of records in the dataset
records_to_fetch = 20000  # Number of records we want to sample
batch_size = 1000  # Number of records to fetch per request

samples = []

for _ in range(records_to_fetch // batch_size):
    offset = random.randint(0, total_records - batch_size)
    results = client.get("qgea-i56i", limit=batch_size, offset=offset)
    samples.extend(results)

df = pd.DataFrame.from_records(samples)


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   cmplnt_num                   20000 non-null  object
 1   cmplnt_fr_dt                 20000 non-null  object
 2   cmplnt_fr_tm                 20000 non-null  object
 3   cmplnt_to_dt                 18586 non-null  object
 4   cmplnt_to_tm                 20000 non-null  object
 5   addr_pct_cd                  19996 non-null  object
 6   rpt_dt                       20000 non-null  object
 7   ky_cd                        20000 non-null  object
 8   ofns_desc                    20000 non-null  object
 9   pd_cd                        19982 non-null  object
 10  pd_desc                      20000 non-null  object
 11  crm_atpt_cptd_cd             20000 non-null  object
 12  law_cat_cd                   20000 non-null  object
 13  boro_nm                      20

In [12]:
#view columns before dropping
df.columns

Index(['cmplnt_num', 'cmplnt_fr_dt', 'cmplnt_fr_tm', 'cmplnt_to_dt',
       'cmplnt_to_tm', 'addr_pct_cd', 'rpt_dt', 'ky_cd', 'ofns_desc', 'pd_cd',
       'pd_desc', 'crm_atpt_cptd_cd', 'law_cat_cd', 'boro_nm',
       'loc_of_occur_desc', 'prem_typ_desc', 'juris_desc', 'jurisdiction_code',
       'parks_nm', 'hadevelopt', 'housing_psa', 'x_coord_cd', 'y_coord_cd',
       'susp_age_group', 'susp_race', 'susp_sex', 'latitude', 'longitude',
       'lat_lon', 'patrol_boro', 'station_name', 'vic_age_group', 'vic_race',
       'vic_sex', ':@computed_region_efsh_h5xi', ':@computed_region_f5dn_yrer',
       ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q',
       ':@computed_region_sbqj_enih', 'transit_district'],
      dtype='object')

In [13]:
# Drop columns that are not needed
df.drop(columns=[":@computed_region_efsh_h5xi",
                  ":@computed_region_f5dn_yrer", 
                  ":@computed_region_yeji_bk3q", 
                  ":@computed_region_92fq_4b7q", 
                  ":@computed_region_sbqj_enih", 
                  "station_name",
                  "latitude",
                  "longitude",
                  "lat_lon",
                  'x_coord_cd',
                  'y_coord_cd',
                  'transit_district',
                  'pd_cd',
                  'ky_cd',
                  'cmplnt_num'])


,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_dt,cmplnt_to_tm,addr_pct_cd,rpt_dt,ofns_desc,pd_desc,crm_atpt_cptd_cd,law_cat_cd,...,parks_nm,hadevelopt,housing_psa,susp_age_group,susp_race,susp_sex,patrol_boro,vic_age_group,vic_race,vic_sex
0,2015-12-06T00:00:00.000,21:45:00,2015-12-06T00:00:00.000,21:59:00,67,2015-12-06T00:00:00.000,DANGEROUS DRUGS,"MARIJUANA, POSSESSION 4 & 5",COMPLETED,MISDEMEANOR,...,(null),(null),(null),<18,BLACK,M,PATROL BORO BKLYN SOUTH,UNKNOWN,UNKNOWN,E
1,2015-12-05T00:00:00.000,10:25:00,2015-12-06T00:00:00.000,01:16:00,14,2015-12-06T00:00:00.000,PETIT LARCENY,"LARCENY,PETIT FROM STORE-SHOPL",COMPLETED,MISDEMEANOR,...,(null),(null),(null),(null),(null),(null),PATROL BORO MAN SOUTH,UNKNOWN,UNKNOWN,D
2,2015-12-05T00:00:00.000,17:00:00,2015-12-05T00:00:00.000,17:30:00,34,2015-12-06T00:00:00.000,BURGLARY,"BURGLARY,RESIDENCE,DAY",COMPLETED,FELONY,...,(null),(null),(null),UNKNOWN,UNKNOWN,U,PATROL BORO MAN NORTH,25-44,UNKNOWN,F
3,2015-12-06T00:00:00.000,02:45:00,2015-12-06T00:00:00.000,02:50:00,43,2015-12-06T00:00:00.000,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",COMPLETED,FELONY,...,(null),(null),887,UNKNOWN,UNKNOWN,U,PATROL BORO BRONX,25-44,BLACK,F
4,2015-12-06T00:00:00.000,21:00:00,2015-12-06T00:00:00.000,21:30:00,66,2015-12-06T00:00:00.000,CRIMINAL MISCHIEF & RELATED OF,"MISCHIEF, CRIMINAL 4, OF MOTOR",COMPLETED,MISDEMEANOR,...,(null),(null),(null),UNKNOWN,UNKNOWN,U,PATROL BORO BKLYN SOUTH,45-64,WHITE,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2022-10-19T00:00:00.000,15:30:00,2022-10-19T00:00:00.000,15:35:00,41,2022-10-20T00:00:00.000,THEFT-FRAUD,"FRAUD,UNCLASSIFIED-FELONY",COMPLETED,FELONY,...,(null),(null),(null),(null),(null),(null),PATROL BORO BRONX,25-44,WHITE HISPANIC,M
19996,2022-10-17T00:00:00.000,18:40:00,2022-10-17T00:00:00.000,18:45:00,112,2022-10-20T00:00:00.000,ROBBERY,"ROBBERY,OPEN AREA UNCLASSIFIED",COMPLETED,FELONY,...,(null),(null),(null),25-44,UNKNOWN,M,PATROL BORO QUEENS NORTH,25-44,ASIAN / PACIFIC ISLANDER,M
19997,2022-09-19T00:00:00.000,00:00:00,2022-10-20T00:00:00.000,00:00:00,50,2022-10-20T00:00:00.000,GRAND LARCENY,"LARCENY,GRAND FROM OPEN AREAS, UNATTENDED",COMPLETED,FELONY,...,(null),(null),(null),UNKNOWN,UNKNOWN,U,PATROL BORO BRONX,25-44,WHITE,M
19998,2022-10-20T00:00:00.000,13:00:00,2022-10-20T00:00:00.000,13:18:00,103,2022-10-20T00:00:00.000,OFFENSES AGAINST PUBLIC ADMINI,RESISTING ARREST,COMPLETED,MISDEMEANOR,...,(null),(null),(null),25-44,ASIAN / PACIFIC ISLANDER,M,PATROL BORO QUEENS SOUTH,UNKNOWN,UNKNOWN,E
